In [1]:
import pandas as pd

In [2]:
# Cargar los datasets
df1 =pd.read_csv(r'DATASETS\_ETL_COUSERA_REVIEWS.csv',encoding='utf-8',sep='|')
df2 =pd.read_csv(r'DATASETS\_ETL_COUSERA.csv',encoding='utf-8',sep='|')
df3=pd.read_csv(r'DATASETS\_ETL_EDX.csv',encoding='utf-8',sep='|')
df4=pd.read_csv(r'DATASETS\_UDEMY_EDX.csv', encoding='utf-8',sep='|')

df_coursera=pd.DataFrame(columns=['title'])
df_edx=pd.DataFrame(columns=['title'])
df_udemy=pd.DataFrame(columns=['title'])

In [3]:
df4.head(5)

,course_id,course_title,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,date_published,subject,language
0,1070968,Ultimate Investment Banking Course,True,200,2147,23,51,All Levels,1.5,2017-01-18,Business Finance,English
1,1113822,Complete GST Course & Certification - Grow You...,True,75,2792,923,274,All Levels,39.0,2017-03-09,Business Finance,English
2,1006314,Financial Modeling for Business Analysts and C...,True,45,2174,74,51,Intermediate Level,2.5,2016-12-19,Business Finance,English
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,True,95,2451,11,36,All Levels,3.0,2017-05-30,Business Finance,English
4,1011058,How To Maximize Your Profits Trading Options,True,200,1276,45,26,Intermediate Level,2.0,2016-12-13,Business Finance,English


In [4]:
df1.head(3)

,reviews,reviewers,date_reviews,rating,course_id
0,pretty dry but i was able to pass with just tw...,By Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,information was perfect the program itself was...,By Jacob D,2020-04-08,4,google-cbrs-cpi-training


In [5]:
df2.head(3)

,name,institution,course_id,language
0,Machine Learning,Stanford University,machine-learning,English
1,Indigenous Canada,University of Alberta,indigenous-canada,Spanish
2,The Science of Well-Being,Yale University,the-science-of-well-being,English


Unificamos el dataset de coursera

In [6]:
dfaux=pd.DataFrame(columns=['course_id'])
dfaux['course_id']=df1['course_id'].unique()

dfaux= dfaux.merge(df1.groupby('course_id')['reviews'].count().reset_index(), on='course_id', how='left')
dfaux= dfaux.merge(df1.groupby('course_id')['reviewers'].count().reset_index(), on='course_id', how='left')
dfaux = dfaux.merge(df1.groupby('course_id')['rating'].mean().round(1).reset_index(), on='course_id', how='left')

dfaux.columns = ['course_id', 'total_reviews', 'total_reviewers', 'average_rating']

courserakpi_df=pd.merge(df2, dfaux, left_on='course_id', right_on='course_id')
courserakpi_df=courserakpi_df.reindex(columns=['course_id','name','institution','language','total_reviews','total_reviewers','average_rating'])
courserakpi_df.to_csv('DATASETS\_KPI_COURSERA.csv',index=False, encoding='utf-8',sep='|')

KPI Tasa de conversión de inscriptos gratuitos a inscriptos pagados

In [7]:
# Filtrar los cursos gratuitos y pagados en df4
free_courses = df4[df4['is_paid'] == False]
paid_courses = df4[df4['is_paid'] == True]

# Contar el número de inscritos en cursos gratuitos y pagados
total_free_inscriptions = free_courses['num_subscribers'].sum()
total_paid_inscriptions = paid_courses['num_subscribers'].sum()

# Calcular la tasa de conversión
conversion_rate = (total_paid_inscriptions / total_free_inscriptions) * 100

# Imprimir la tasa de conversión
print("Tasa de conversión de inscritos gratuitos a inscritos pagados:", conversion_rate)
print("total_free:", total_free_inscriptions)
print("total_paid:", total_paid_inscriptions)

Tasa de conversión de inscritos gratuitos a inscritos pagados: 227.66754532551167
total_free: 3575525
total_paid: 8140310


La tasa de conversión de inscritos gratuitos a inscritos pagados es de aproximadamente 227.67. Esto significa que, en promedio, por cada inscrito en cursos gratuitos, hay alrededor de 227.67 inscritos en cursos pagados.

courserakpi_df:['course_id', 'name', 'institution', 'language', 'total_reviews','total_reviewers', 'average_rating']

df4: ['course_id', 'course_title', 'is_paid', 'price', 'num_subscribers', 'num_reviews', 'num_lectures', 'level', 'content_duration', 'date_published', 'subject', 'language']

df3:['title', 'summary', 'n_enrolled', 'course_type', 'institution', 'Level', 'subject', 'language', 'subtitles', 'course_effort', 'course_length', 'price', 'course_description']

Proporción de cursos con subtítulos

In [8]:
# Cálculo de la proporción de cursos con subtítulos
subtitles_ratio = (df3[df3['subtitles'].notnull()].shape[0] / df3.shape[0]) * 100
subtitles_ratio

100.0

Índice de satisfacción del curso: 

In [9]:
# Cálculo del índice de satisfacción del curso
max_rating = courserakpi_df['average_rating'].max()
max_reviews = courserakpi_df['total_reviews'].max()
satisfaction_index = (courserakpi_df['average_rating'] * courserakpi_df['total_reviews']) / (max_rating * max_reviews)
satisfaction_index

0      0.782446
1      0.092308
2      0.528338
3      0.761918
4      0.002086
         ...   
599    0.036724
600    0.006224
601    0.000369
602    0.024721
603    0.017396
Length: 604, dtype: float64

## KPI 1 - Tasa de finalizacion de Cursos

Esta métrica representa el porcentaje de estudiantes que completaron un curso en relación con el número total de inscritos. Puedes calcularlo dividiendo el número de estudiantes que completaron el curso entre el número total de inscritos y multiplicarlo por 100. Esto te ayudará a evaluar la efectividad de tus cursos para retener y motivar a los estudiantes a completarlos.

In [10]:
# Cálculo de la tasa de finalización de cursos
completion_rate = (df4['num_reviews'] / df4['num_subscribers']) * 100

In [11]:
completion_rate

0        1.071262
1       33.058739
2        3.403864
3        0.448796
4        3.526646
          ...    
3667     1.346154
3668     0.980392
3669    32.943470
3670    10.333333
3671     3.995560
Length: 3672, dtype: float64

## KPI 2 - Valoración promedio de los cursos

Esta métrica se refiere a la calificación promedio dada por los estudiantes a los cursos. Puedes calcularlo tomando el promedio de la columna 'average_rating' en el dataset courserakpi_df. Esto te dará una idea de la satisfacción general de los estudiantes con los cursos ofrecidos.

In [12]:
# Cálculo de la valoración promedio de los cursos
average_rating = courserakpi_df['average_rating'].mean()

In [13]:
average_rating

4.619867549668874

## KPI 3 - Crecimiento de suscripciones

Esta métrica indica la tasa de crecimiento de las suscripciones a lo largo del tiempo. Puedes calcularlo comparando el número de suscriptores en una fecha específica con el número de suscriptores en una fecha anterior y expresarlo como un porcentaje de crecimiento.

In [14]:
# Cálculo del crecimiento de suscripciones
growth_rate = ((df4['num_subscribers'].iloc[-1] - df4['num_subscribers'].iloc[0]) / df4['num_subscribers'].iloc[0]) * 100

In [15]:
growth_rate

-58.03446669771775